<center>
<a href="https://github.com/kamu-data/kamu-cli">
<img alt="kamu" src="https://raw.githubusercontent.com/kamu-data/kamu-cli/master/docs/readme_files/kamu_logo.png" width=270/>
</a>
</center>

<br/>

<div align="center">
<a href="https://github.com/kamu-data/kamu-cli">Repo</a> | 
<a href="https://docs.kamu.dev/cli/">Docs</a> | 
<a href="https://docs.kamu.dev/cli/learn/learning-materials/">Tutorials</a> | 
<a href="https://docs.kamu.dev/cli/learn/examples/">Examples</a> |
<a href="https://docs.kamu.dev/cli/get-started/faq/">FAQ</a> |
<a href="https://discord.gg/nU6TXRQNXC">Discord</a> |
<a href="https://kamu.dev">Website</a>
</div>


<center>

<br/>
    
# KPI Metrics Pipeline
## Github Metrics

</center>

Welcome to the KPI metrics demo series. If you're new to Kamu and haven't worked on it before, we suggest you to go through the [Introduction] demo to understand the basics. You can also visit [Installation] page, and try Kamu locally.

This environment comes with a pre-installed Kamu environment, so no need to install to start working on this online demo.

In the first part of the demo, we're going to create pipelines to gather "views", "stargazers", "subscribers" and "clones" metrics using [Github REST API](https://docs.github.com/en/rest?apiVersion=2022-11-28).

We're going to use two of Kamu's github repositories as examples: [Kamu](https://github.com/kamu-data/kamu-cli) tool and our [Open Data Fabric](http://opendatafabric.org/) protocol. You can change the repo address to get your own KPI pipeline. When we're done you're going to have a pipeline that looks similar to this:



### Initiating Workspace

Any Kamu project starts by creating a workspace, which is a directory where `kamu` stores data and metadata of the datasets.

<div class="alert alert-block alert-success">
Let's create the workspace for this project (we'll do it right in the home directory):<br/>

<p style="background:black">
<code style="background:black;color:white">cd "???????????????????????????????"
kamu init
</code>
</p>
</div>

### Getting Repository Traffic Data
Github stores [traffic](https://docs.github.com/en/rest/metrics/traffic?apiVersion=2022-11-28) information for all repositories in a 14-day window. This include page views, referrals and clones. This creates a perfect use-case for Kamu, as you can set Kamu to automatically pull data from Github every 14 days!

Let's start with "clones", Github suggest to use the following cURL command:

<p style="background:black">
<code style="background:black;color:white">
curl -L \
  -H "Accept: application/vnd.github+json" \
  -H "Authorization: Bearer <YOUR-TOKEN>" \
  -H "X-GitHub-Api-Version: 2022-11-28" \
  https://api.github.com/repos/OWNER/REPO/traffic/clones
</code>
</p>
</div>

We'll create a dataset definition in a .yaml file that includes parts of this information.
<div class="alert alert-block alert-success">
To add the dataset definition to the workspace run:<br/>
<p style="background:black">
<code style="background:black;color:white">kamu add datasets/com.github/kamu-cli-github-clones.yaml
</code>
</p>
</div>

The dataset definition in the .yaml file looks like the following:
```
kind: DatasetSnapshot
version: 1
content:
  name: com.github.kamu-cli.clones
  kind: Root
  metadata:
    - kind: SetPollingSource
      fetch:
        kind: Url
        url: https://api.github.com/repos/kamu-data/kamu-cli/traffic/clones
        headers:
          - name: Accept
            value: application/vnd.github+json
          - name: Authorization
            value: Bearer <INSERT_GITHUB_TOKEN>
...
```
The beginning section mentions this is a **root** dataset, 


.yaml file continues:
```
...
      read:
        kind: Json
        subPath: clones
        schema:
          - timestamp TIMESTAMP
          - count BIGINT
          - uniques BIGINT
      preprocess:
        kind: Sql
        engine: datafusion
        query: |
          SELECT
            timestamp as event_time,
            count,
            uniques as unique_count
          FROM input
...
```

And finally the dataset definition ends with the **merge strategy**:

```
...
      merge:
        kind: Ledger
        primaryKey:
          - event_time
    - kind: SetInfo
      description: Two weeks of clones of the selected github repository.
```

This file can also include further information about 

<div class="alert alert-block alert-warning">
<details>
<summary style="display:list-item"><b>You can also create a .yaml file from a template!</b></summary>

Run the code below:
<p style="background:black">
<code style="background:black;color:white">kamu new --root com.github.kamu-cli.clones
</code>
</p>

This will create a **template** .yaml file for a root dataset definition. It includes comments in each section to create a dataset. Follow directions in the file's comments and use `kamu add com.github.kamu-cli.clones.yaml` when ready.

</details>
</div>

Now github stars are a bit more tricky to achieve, as it includes pagination. For that, we will create an ingest script